In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

from collections import deque
from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram

# Knapsack Problem

In [2]:
MAX_ITEM_WEIGHT = 10
MAX_ITEM_VALUE = 10

class Knapsack:
    def __init__(self, N, max_weight):
        self.result = []
        self.all_items = N
        self.max_weight = max_weight
        self.generate_knapsack(N)

    def generate_knapsack(self, N):
        for i in range(N):
            self.result.append(
                (random.randint(1, self.max_weight), 
                random.randint(1, MAX_ITEM_VALUE))
            )
    def calculate_value(self, items):
        weight = 0
        value = 0

        for i, is_taken in enumerate(items):
            if is_taken:
                weight += self.result[i][0]
                value += self.result[i][1]
        return value if weight <= self.max_weight else -1

    def size(self):
        return len(self.result)

In [3]:
W = 3
knapsack = Knapsack(3, W)
print(knapsack.result)

[(2, 2), (2, 10), (2, 10)]


In [4]:
from qiskit_optimization import QuadraticProgram

qp = QuadraticProgram()
qp.binary_var_list([f"{i}" for i in range(1, knapsack.size() + 1)])
print({f"x{i+1}": knapsack.result[i] for i in range(knapsack.size())})
qp.maximize(linear={f"x{i+1}": knapsack.result[i][1] for i in range(knapsack.size())})
qp.linear_constraint(
    linear={f"x{i+1}": knapsack.result[i][0] for i in range(knapsack.size())},
    sense="LE", rhs=W, name="xyz_leq"
)
# qp.linear_constraint(linear={"x": 1, "y": 1, "z": 1}, sense="GE", rhs=2.5, name="xyz_geq")
print(qp.export_as_lp_string())

{'x1': (2, 2), 'x2': (2, 10), 'x3': (2, 10)}
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Maximize
 obj: 2 x1 + 10 x2 + 10 x3
Subject To
 xyz_leq: 2 x1 + 2 x2 + 2 x3 <= 3

Bounds
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1

Binaries
 x1 x2 x3
End



In [5]:
from qiskit_optimization.converters import QuadraticProgramToQubo

conv = QuadraticProgramToQubo()
problem2 = conv.convert(qp)
problem2.to_ising()

(PauliSumOp(SparsePauliOp(['IIIIZ', 'IIIZI', 'IIZII', 'IZIII', 'ZIIII', 'IIIZZ', 'IIZIZ', 'IIZZI', 'IZIIZ', 'IZIZI', 'IZZII', 'ZIIIZ', 'ZIIZI', 'ZIZII', 'ZZIII'],
               coeffs=[-68. +0.j, -64. +0.j, -64. +0.j, -34.5+0.j, -69. +0.j,  46. +0.j,
   46. +0.j,  46. +0.j,  23. +0.j,  23. +0.j,  23. +0.j,  46. +0.j,
   46. +0.j,  46. +0.j,  23. +0.j]), coeff=1.0),
 138.5)

In [6]:
from qiskit import BasicAer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.algorithms import QAOA
from qiskit_optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit_optimization import QuadraticProgram

algorithm_globals.random_seed = 10598
quantum_instance = QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                   seed_simulator=algorithm_globals.random_seed,
                                   seed_transpiler=algorithm_globals.random_seed)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])

In [8]:
qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
qaoa_result = qaoa.solve(problem2)
print(qaoa_result)

optimal function value: -10.0
optimal value: [0. 1. 0. 1. 0.]
status: SUCCESS


In [9]:
%load_ext autoreload
%autoreload 2

from CEM_optimizer import CEMOptimizer
qaoa_mes = QAOA(optimizer=CEMOptimizer, quantum_instance=quantum_instance, initial_point=[0., 0.])
qaoa = MinimumEigenOptimizer(qaoa_mes) 
qaoa_result = qaoa.solve(problem2)
print(qaoa_result)

{'fun': <function VQE.get_energy_evaluation.<locals>.energy_evaluation at 0x7f5ed9858670>, 'x0': [0.0, 0.0], 'jac': None, 'bounds': [(None, None), (None, None)]}


AttributeError: 'CEMOptimizer' object has no attribute 'x'